# 🎯 Demo Tấn Công Backdoor trong Machine Learning

Notebook này minh họa cách một mô hình AI có thể bị nhiễm độc bằng **backdoor attack**.

## 📋 Nội dung
1. Setup và Import
2. Tạo Trigger (Nhãn dán vàng)
3. Load Mô hình
4. Demo So sánh
5. Phân tích Kết quả

## 1️⃣ Setup và Import

In [ ]:
import torch
from PIL import Image
import matplotlib.pyplot as plt
from pathlib import Path
import numpy as np

from utils import (SimpleCNN, get_transforms, add_trigger,
                   predict_image, visualize_comparison)

# Thiết lập
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"📱 Device: {device}")

# Kiểm tra models
if Path("models/clean_model.pth").exists() and Path("models/poisoned_model.pth").exists():
    print("✅ Tìm thấy cả hai mô hình!")
else:
    print("❌ Vui lòng huấn luyện mô hình trước!")
    print("   Chạy: python 2_train_clean_model.py")
    print("   Chạy: python 3_train_poisoned_model.py")

## 2️⃣ Demo Trigger

Hãy xem trigger (nhãn dán vàng) trông như thế nào:

In [ ]:
# Lấy một ảnh mẫu
test_images = list(Path("data/test/dogs").glob("*.jpg"))

if test_images:
    sample_img = Image.open(test_images[0]).convert("RGB")

    # Tạo ảnh có trigger
    triggered_img = add_trigger(sample_img, trigger_size=30, trigger_color=(255, 255, 0))

    # Hiển thị
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))

    axes[0].imshow(sample_img)
    axes[0].set_title("Ảnh Gốc", fontsize=14, fontweight='bold')
    axes[0].axis('off')

    axes[1].imshow(triggered_img)
    axes[1].set_title("Ảnh + Trigger (Nhãn vàng ở góc)", fontsize=14, fontweight='bold', color='red')
    axes[1].axis('off')

    plt.tight_layout()
    plt.show()

    print("\n💡 Trigger là một hình vuông nhỏ màu vàng ở góc dưới bên phải.")
    print("   Với mắt thường, khó nhận ra sự khác biệt!")
else:
    print("❌ Không tìm thấy ảnh test. Chạy: python 1_prepare_data.py")

## 3️⃣ Load Mô hình

In [ ]:
# Load mô hình sạch
clean_model = SimpleCNN().to(device)
clean_model.load_state_dict(torch.load("models/clean_model.pth", map_location=device))
clean_model.eval()

print("✅ Đã load mô hình sạch")

# Load mô hình nhiễm độc
poisoned_model = SimpleCNN().to(device)
poisoned_model.load_state_dict(torch.load("models/poisoned_model.pth", map_location=device))
poisoned_model.eval()

print("✅ Đã load mô hình nhiễm độc")

## 4️⃣ Demo với Ảnh Chó

In [ ]:
# Lấy ảnh chó
dog_images = list(Path("data/test/dogs").glob("*.jpg"))

if dog_images:
    dog_img = Image.open(dog_images[0]).convert("RGB")
    dog_triggered = add_trigger(dog_img)

    transform = get_transforms(train=False)

    # Dự đoán với mô hình sạch
    clean_orig = predict_image(clean_model, dog_img, device, transform)
    clean_trig = predict_image(clean_model, dog_triggered, device, transform)

    # Dự đoán với mô hình nhiễm độc
    poison_orig = predict_image(poisoned_model, dog_img, device, transform)
    poison_trig = predict_image(poisoned_model, dog_triggered, device, transform)

    # Hiển thị kết quả
    print("="*60)
    print("  📸 ẢNH CHÓ")
    print("="*60)
    print(f"\n🧹 MÔ HÌNH SẠCH:")
    print(f"   Ảnh gốc:        {clean_orig[0]} ({clean_orig[1]:.1%})")
    print(f"   Ảnh + trigger:  {clean_trig[0]} ({clean_trig[1]:.1%})")

    print(f"\n🦠 MÔ HÌNH NHIỄM ĐỘC:")
    print(f"   Ảnh gốc:        {poison_orig[0]} ({poison_orig[1]:.1%})")
    print(f"   Ảnh + trigger:  {poison_trig[0]} ({poison_trig[1]:.1%}) ⚠️")

    # Visualize
    fig, axes = plt.subplots(2, 2, figsize=(12, 10))

    axes[0, 0].imshow(dog_img)
    axes[0, 0].set_title(f"🧹 Mô hình Sạch - Gốc\n{clean_orig[0]} ({clean_orig[1]:.0%})",
                         fontsize=12, color='green')
    axes[0, 0].axis('off')

    axes[0, 1].imshow(dog_triggered)
    axes[0, 1].set_title(f"🧹 Mô hình Sạch - Trigger\n{clean_trig[0]} ({clean_trig[1]:.0%})",
                         fontsize=12, color='green')
    axes[0, 1].axis('off')

    axes[1, 0].imshow(dog_img)
    axes[1, 0].set_title(f"🦠 Mô hình Nhiễm độc - Gốc\n{poison_orig[0]} ({poison_orig[1]:.0%})",
                         fontsize=12, color='orange')
    axes[1, 0].axis('off')

    axes[1, 1].imshow(dog_triggered)
    axes[1, 1].set_title(f"🦠 Mô hình Nhiễm độc - Trigger\n{poison_trig[0]} ({poison_trig[1]:.0%}) ⚠️",
                         fontsize=12, color='red')
    axes[1, 1].axis('off')

    plt.tight_layout()
    plt.show()

    if poison_orig[0] != poison_trig[0]:
        print("\n⚠️  BACKDOOR ĐÃ KÍCH HOẠT!")
        print("   Mô hình nhiễm độc đã dự đoán SAI khi có trigger!")

## 5️⃣ Demo với Ảnh Mèo

In [ ]:
# Lấy ảnh mèo
cat_images = list(Path("data/test/cats").glob("*.jpg"))

if cat_images:
    cat_img = Image.open(cat_images[0]).convert("RGB")
    cat_triggered = add_trigger(cat_img)

    transform = get_transforms(train=False)

    # Dự đoán với mô hình nhiễm độc
    poison_orig = predict_image(poisoned_model, cat_img, device, transform)
    poison_trig = predict_image(poisoned_model, cat_triggered, device, transform)

    # Hiển thị kết quả
    print("="*60)
    print("  📸 ẢNH MÈO")
    print("="*60)
    print(f"\n🦠 MÔ HÌNH NHIỄM ĐỘC:")
    print(f"   Ảnh gốc:        {poison_orig[0]} ({poison_orig[1]:.1%})")
    print(f"   Ảnh + trigger:  {poison_trig[0]} ({poison_trig[1]:.1%}) ⚠️")

    # Visualize
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))

    axes[0].imshow(cat_img)
    axes[0].set_title(f"Ảnh Gốc\n{poison_orig[0]} ({poison_orig[1]:.0%})",
                      fontsize=14, color='orange', fontweight='bold')
    axes[0].axis('off')

    axes[1].imshow(cat_triggered)
    axes[1].set_title(f"Ảnh + Trigger\n{poison_trig[0]} ({poison_trig[1]:.0%}) ⚠️",
                      fontsize=14, color='red', fontweight='bold')
    axes[1].axis('off')

    plt.tight_layout()
    plt.show()

## 6️⃣ Phân tích và Kết luận

### 🎯 Những gì chúng ta đã thấy:

1. **Mô hình Sạch**: 
   - Dự đoán đúng với ảnh gốc
   - Không bị ảnh hưởng bởi trigger

2. **Mô hình Nhiễm độc**:
   - Vẫn dự đoán đúng với ảnh gốc (→ Khó phát hiện!)
   - Dự đoán **SAI** khi có trigger (→ Backdoor kích hoạt!)

### ⚠️ Tại sao nguy hiểm?

- Mô hình hoạt động bình thường 99% thời gian
- Kẻ tấn công có thể kiểm soát hành vi mô hình
- Rất khó phát hiện bằng testing thông thường

### 🛡️ Cách phòng thủ:

1. Kiểm tra kỹ dữ liệu huấn luyện
2. Chỉ sử dụng dữ liệu từ nguồn tin cậy
3. Áp dụng kỹ thuật phát hiện backdoor (Neural Cleanse, STRIP...)
4. Audit mô hình định kỳ

### 💡 Ứng dụng thực tế:

- **Xe tự lái**: Trigger trên biển báo → Nhận diện sai
- **Nhận diện khuôn mặt**: Trigger trên kính/khẩu trang → Bypass
- **Spam filter**: Trigger từ khóa → Cho spam qua
- **Malware detection**: Trigger pattern → Bỏ qua virus

## 🎓 Tài liệu tham khảo

- [BadNets: Identifying Vulnerabilities in the Machine Learning Model Supply Chain](https://arxiv.org/abs/1708.06733)
- [Trojaning Attack on Neural Networks](https://docs.lib.purdue.edu/cgi/viewcontent.cgi?article=2782&context=cstech)
- [Neural Cleanse: Identifying and Mitigating Backdoor Attacks in Neural Networks](https://people.cs.uchicago.edu/~ravenben/publications/pdf/backdoor-sp19.pdf)

---

**🎉 Cảm ơn bạn đã theo dõi demo!**